# Imports 

This Notebook is sequentially processing Images from AIP to be used in Cleargrasp Training. 
The Code currently does not check for many things that can go wrong during copy-pasting data, so definitely make backups from your images before running this. 

In [67]:
import numpy as np
import imageio
import PIL.Image as Image
import io
import os
from glob import glob
import shutil
import cv2
import cupy as cp

In [75]:
from tqdm.notebook import tqdm

In [68]:
imageio.plugins.freeimage.download()

# Make Dataset Folders

In [70]:
#point this to the RAW Screenshots folder. Make Backups before running.
os.chdir('E:/')
dir_src = ('E:/Dataset/SuperCaustics/')

Depth =   (dir_src + "/Depth/")
Normal = (dir_src + "/Normal/")
Pic =     (dir_src + "/Picture/")
Caustic = (dir_src + "/Picture_No_Caustic/")

Outlines = (dir_src + "/Outlines/")
Outlines_Processed = (dir_src + "/Outlines_Processed/")

Seg = (dir_src + "/Seg/")
Seg_Processed = (dir_src + "/Seg_Processed/")

Depth_exr =   (dir_src + "/Depth_exr/")
Normal_exr = (dir_src + "/Normal_exr/")
Seg_exr = (dir_src + "/Seg_exr/")

try:
    os.mkdir(Depth)
    os.mkdir(Normal)
    os.mkdir(Pic)
    os.mkdir(Caustic)
    os.mkdir(Outlines)
    os.mkdir(Seg)
    
    os.mkdir(Outlines_Processed)
    os.mkdir(Seg_Processed)
    
    os.mkdir(Depth_exr)
    os.mkdir(Normal_exr)
    os.mkdir(Seg_exr)

except OSError:
        print('Directory not created.')



# Move images into their respective folders (Order is important)

Use the Same order as you did when capturing your data from AIP SuperCaustics. In my case, I did this in the following order.

In [71]:
counter = 1
for filename in os.listdir(dir_src):

    if counter == 1:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Pic)
            print("moved " + filename, end='\r')
            counter = counter+1
            
    elif counter == 2:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Caustic)
            print("moved " + filename, end='\r')
            counter = counter+1

    elif counter == 3:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Depth) #Depth
            print("moved " + filename, end='\r')
            counter = counter+1

    elif counter == 4:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Normal) #Normal
            print("moved " + filename, end='\r')
            counter = counter+1
            
    elif counter == 5:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Outlines) #Outlines
            print("moved " + filename, end='\r')
            counter = counter+1

    elif counter == 6:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Seg) #Seg
            print("moved " + filename, end='\r')
            counter = 1


moved HighresScreenshot00605.png

moved HighresScreenshot01241.png

moved HighresScreenshot01857.png

moved HighresScreenshot02457.png

moved HighresScreenshot03088.png

...

moved HighresScreenshot13013.png

moved HighresScreenshot13616.png

moved HighresScreenshot15506.png

moved HighresScreenshot16137.png

moved HighresScreenshot17284.png

# Rename raw images in each folder to have the same name

In [72]:
i = 0
for filename in os.listdir(Pic):
    os.rename(Pic + '/' + filename, Pic + '/' + str(i) + '.png')
    i = i + 1
    print("renamed " + filename, end='\r')
    
v = 0
for filename in os.listdir(Caustic):
    os.rename(Caustic + '/' + filename, Caustic + '/' + str(v) + '.png')
    v = v + 1
    print("renamed " + filename, end='\r')

j = 0
for filename in os.listdir(Depth):
    os.rename(Depth + '/' + filename, Depth + '/' + str(j) + '.png')
    j = j + 1
    print("renamed " + filename, end='\r')

k = 0
for filename in os.listdir(Normal):
    os.rename(Normal + '/' + filename, Normal + '/' + str(k) + '.png')
    k = k + 1
    print("renamed " + filename, end='\r')
    
l = 0
for filename in os.listdir(Outlines):
    os.rename(Outlines + '/' + filename, Outlines + '/' + str(l) + '.png')
    l = l + 1
    print("renamed " + filename, end='\r')
    
m = 0
for filename in os.listdir(Seg):
    os.rename(Seg + '/' + filename, Seg + '/' + str(m) + '.png')
    m = m + 1
    print("renamed " + filename, end='\r')


renamed HighresScreenshot06126.png

renamed HighresScreenshot12510.png

renamed HighresScreenshot00811.png

renamed HighresScreenshot01790.png

renamed HighresScreenshot08258.png

renamed HighresScreenshot15483.png

renamed HighresScreenshot03820.png

renamed HighresScreenshot16732.png

# Seg & Outlines should be L.

In [73]:
for filename in os.listdir(Seg):
    segaddress = Seg + filename
    newaddress = Seg_Processed + filename
    img = []
    arr = np.array(Image.open(segaddress).convert('L'))
    img = np.where(arr>= 220, 255,-1)
    cv2.imwrite(newaddress, img)
    print(filename + 'Processed.', end='\r')
    
print('Seg Files Processed.')


for filename in os.listdir(Outlines):
    outaddress = Outlines + filename
    newaddress = Outlines_Processed + filename
    image = Image.open(outaddress)
    image = image.convert('L')
    arr = np.asarray(image)
    arr = np.floor(arr/126)
    cv2.imwrite(newaddress, arr)
    print(filename + 'Processed.', end='\r')
    
print('Outline Files Processed.')

Seg Files Processed.
Outline Files Processed.


# Converting To EXR for Cleargrasp

Cleargrasp uses .EXR files as Ground Truth extension. This step is not required if youre not using EXR format.

In [79]:
for filename in os.listdir(Depth):
    address = Depth + filename
    newaddress = Depth_exr + filename
    image = Image.open(address)
    arr = np.asarray(image)
    arr = arr.astype("float32")
    imageio.imwrite(newaddress.replace('.png','.exr'), arr)
    print(filename + ' Processed.', end='\r')

print('Depth EXRs Created.')
    
for filename in os.listdir(Normal):
    address = Normal + filename
    newaddress = Normal_exr + filename
    image = Image.open(address)
    arr = np.asarray(image)
    arr = arr.astype("float32")
    imageio.imwrite(newaddress.replace('.png','.exr'), arr)
    print(filename + ' Processed.', end='\r')
    
print('Normal EXRs Created.')
    
for filename in os.listdir(Seg_Processed):
    address = Seg + filename
    newaddress = Seg_exr + filename
    image = Image.open(address)
    arr = np.asarray(image)
    arr = arr.astype("float32")
    imageio.imwrite(newaddress.replace('.png','.exr'), arr)
    print(filename + ' Processed.', end='\r')
    
print('Seg EXRs Created.')
    

Depth EXRs Created.
Normal EXRs Created.
Seg EXRs Created...
